In [ ]:
import json
import os.path as osp
import pandas as pd

from cheffelo_personalization.basket_preselector.main import run_preselector_batch, run_preselector_batch_api
from cheffelo_personalization.basket_preselector.data.processor import process_api_data, process_raw_data, parse_recipe_information
from cheffelo_personalization.basket_preselector.utils.paths import TEST_DATA_DIR
from cheffelo_personalization.basket_preselector.data.customers import (
    get_customers_with_mealbox,
)
from cheffelo_personalization.basket_preselector.utils.constants import (
    FLEX_PRODUCT_TYPE_ID,
    MEALBOX_PRODUCT_TYPE_ID,
)

In [ ]:
def open_file(fn):
    with open(fn) as f:
        data = json.load(f)
    return data

In [ ]:
agreements = open_file(osp.join(TEST_DATA_DIR, "agreements.json"))
product_information = open_file(osp.join(TEST_DATA_DIR, "product_information.json"))
menu_products = open_file(osp.join(TEST_DATA_DIR, "menu_products.json"))
recommendation_scores = open_file(osp.join(TEST_DATA_DIR, "recommendation_scores.json"))
preference_rules = open_file(osp.join(TEST_DATA_DIR, "product_preference_rules.json"))

In [ ]:
results = run_preselector_batch_api(
    agreements,
    product_information,
    menu_products,
    recommendation_scores,
    preference_rules
)

In [ ]:
results

In [ ]:
menu_products

In [ ]:
df_products = pd.DataFrame(product_information)
df_recipes = parse_recipe_information(recipe_information=menu_products)
df_preference_rules = pd.DataFrame(preference_rules)

df_menu = df_recipes.merge(df_products, on="variation_id", how="left").rename(
    columns={
        "preferences": "preference_ids",
        "product_id": "product_id",
        "name": "variation",
        "portions": "variation_portions",
        "meals": "variation_meals",
        "price": "variation_price",
        "main_recipe_ids": "main_recipe_id",
    }
)
df_menu = df_menu.assign(
    product_type_id=df_menu["product_type_id"].apply(lambda s: s.upper())
)

df_customers = pd.DataFrame(agreements)

In [ ]:
df_menu.head()

In [ ]:
df_customers.head()

In [ ]:
df_mealboxes = df_menu[
    df_menu["product_type_id"] == MEALBOX_PRODUCT_TYPE_ID.upper()
][
    [
        "product_id",
        "variation",
        "variation_id",
        "variation_portions",
        "variation_meals",
        "variation_price",
    ]
].drop_duplicates()
df_mealboxes.head()

In [ ]:
df_flex_products = df_menu[
    df_menu["product_type_id"] == FLEX_PRODUCT_TYPE_ID.upper()
]
df_flex_products.head()